<a href="https://colab.research.google.com/github/AdryGP/ABP-FindMo/blob/main/GoodreadsWS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Uso de API goodreads para python

Primero es necesario instalar goodreads en la maquina

In [ ]:
!pip install goodreads

     |████████████████████████████████| 163kB 4.3MB/s 
  Created wheel for goodreads: filename=goodreads-0.3.2-cp36-none-any.whl size=13117 sha256=102dbdbf5d76d361a7d5fa1bfff0078a03993d23b77b33ed53c1e35f55e5ba01
  Stored in directory: /root/.cache/pip/wheels/70/10/60/95c53185e3b96fa8da290cefe495d5eef298734b7d7f634e72
  Created wheel for rauth: filename=rauth-0.7.3-cp36-none-any.whl size=16055 sha256=4d0e09c8d4d446e86a1c77fc30ece171d48f782fdf36167811f454e2821d0267
  Stored in directory: /root/.cache/pip/wheels/7b/94/5d/81afc278dd5da884a0002563dc4b0fe85f9067a5a40f76f858
Successfully built goodreads rauth


Se comprueba que esta instalado con exito

In [ ]:
!pip list -v | grep [Gg]oodreads

goodreads                     0.3.2           /usr/local/lib/python3.6/dist-packages pip      


Una vez goodreads esta instalado correctamente, es necesario añadir la clave de desarrollador para poder realizar la gran mayoria de acciones de la API

In [ ]:
from goodreads import client
gc = client.GoodreadsClient('Iz1kiMAny7IclmuSPHlA', 'OKtyleWYG9mBV05NHdsFEuUAB3QrZvvwJeQ6CjHIpY')

Se comprueba que se puede acceder al servidor de Goodreads

In [ ]:
 book = gc.book(10000)
 book.description

'Like an elegantly chilling postscript to <em>The Metamorphosis</em>, this classic of postwar Japanese literature describes a bizarre physical transformation that exposes the duplicities of an entire world. The narrator is a scientist hideously deformed in a laboratory accident–a man who has lost his face and, with it, his connection to other people. Even his wife is now repulsed by him. <br /><br />His only entry back into the world is to create a mask so perfect as to be undetectable. But soon he finds that such a mask is more than a disguise: it is an alternate self–a self that is capable of anything. A remorseless meditation on nature, identity and the social contract, <em>The Face of Another</em> is an intellectual horror story of the highest order.'

Una vez esta todo preparado, se crea el directorio donde acabara el .csv a su salida

In [ ]:
!mkdir -p original_datasets/libros 

A continuacion se conseguiran los datos para crear el csv, es importante comentar varias cosas. 

La primera es que goodreads pide que solo se realice una peticion a su servidor cada segundo.

La seguda es que existen IDs que han sido eliminados de la base de datos, con lo que podemos encontrarnos que al accedera un libro este no exista, esto lanzara una excepcion que debe estar controlada.

La tercera es que hay libros que no tienen descripcion, que seran ignorados ya que no seran interesantes para la aplicacion.

Y la cuarta es que algunas descripciones tienen caracteres que son equivalentes a los de separacion del csv, con lo que los datos se añaden uno a uno para evitar errores a la hora de la escritura, si esto ocurre sera necesaria su edicion de forma externa.



In [ ]:
import pandas as pd
import re
import time

Titulo = list(())
Autor = list(())
Descripcion = list(())
Id = list(())

from csv import writer
def append_list_as_row(file_name, list_of_elem):
    # Open file in append mode
    with open(file_name, 'a+', newline='') as write_obj:
        # Create a writer object from csv module
        csv_writer = writer(write_obj)
        # Add contents of list as last row in the csv file
        csv_writer.writerow(list_of_elem)


try:
    y=1
    book = gc.book(y)
    descripcion=book.description      
    if not descripcion:
      print(y)
    else:
      #Las descripciones viene con cadenas html en su interior, que sera necesario eliminarlas con esta funcion
      descripcionLimpia= re.sub('<.*>', '', descripcion) 
      Descripcion.append(descripcionLimpia)
      Titulo.append(book.title)
      Autor.append(book.authors)
      Id.append('L' + str(1) )

except:
      pass
time.sleep(1)
data = {'Titulo','ID', 'Autor','Descripcion'}
df = pd.DataFrame ( {'Titulo' : Titulo,'ID': Id, 'Autor':Autor,'Descripcion':Descripcion})
df[['Titulo', 'ID', 'Autor','Descripcion',]].to_csv('original_datasets/libros/goodreadsBooks.csv', index=False)

Libro = list(())
data = {'Titulo','ID', 'Autor','Descripcion'}

for x in range(2,10):
  try:
    book = gc.book(x)
    descripcion=book.description      
    if not descripcion:
      print(x)
    else:
      descripcionLimpia= re.sub('<.*>', '', descripcion)
      
      Libro.append(book.title)
      Libro.append('L' + str(x) )
      Libro.append(book.authors)      
      Libro.append(descripcionLimpia)

      append_list_as_row('original_datasets/libros/goodreadsBooks.csv', Libro)
      Libro = list(())
  except:
      pass
  time.sleep(1)
print('finalizado')



finalizado


En caso de modificar de forma externa el csv puede ser necesario eliminar columnas residuales

In [ ]:
libros = pd.read_csv('original_datasets/libros/goodreadsBooks.csv')

Titulo = list(())
Autor = list(())
Descripcion = list(())
Id = list(())
Titulo = libros['Titulo']
Autor = libros['Autor']
Descripcion = libros['Descripcion']
Id = libros['ID']

df = pd.DataFrame ( {'Titulo' : Titulo,'ID': Id, 'Autor':Autor,'Descripcion':Descripcion})
df[['Titulo', 'ID', 'Autor','Descripcion',]].to_csv('original_datasets/libros/goodreadsBooksModified.csv', index=False)



Aunque se controla que las descripciones no sean vacias, puede quedar alguna en la que no hay texto, esto puede ser o bien porque solo tenian codigo html o que tenian espacios o /n

In [ ]:
libros = pd.read_csv('original_datasets/libros/goodreadsBooks.csv')
modifiedcsv = libros.dropna()
modifiedcsv.isnull().sum()
modifiedcsv

,Titulo,ID,Autor,Descripcion
0,Harry Potter and the Half-Blood Prince (Harry ...,L1,[J.K. Rowling],The war against Voldemort is not going well; e...
1,Harry Potter and the Order of the Phoenix (Har...,L2,"[J.K. Rowling, Mary GrandPré]",There is a door at the end of a silent corrido...
2,Harry Potter and the Sorcerer's Stone (Harry P...,L3,"[J.K. Rowling, Mary GrandPré]",Harry Potter's life is miserable. His parents ...
3,Harry Potter and the Chamber of Secrets (Harry...,L4,[J.K. Rowling],The Dursleys were so mean and hideous that sum...
4,Harry Potter and the Prisoner of Azkaban (Harr...,L5,"[J.K. Rowling, Mary GrandPré]","For twelve long years, the dread fortress of A..."
5,Harry Potter and the Goblet of Fire (Harry Pot...,L6,"[J.K. Rowling, Mary GrandPré]",Harry Potter is midway through his training as...
7,"Harry Potter Boxed Set, Books 1-5 (Harry Potte...",L8,"[J.K. Rowling, Mary GrandPré]",Box Set containing Harry Potter and the Sorcer...
8,"Unauthorized Harry Potter Book Seven News: ""Ha...",L9,[W. Frederick Zimmerman],Through the magic of print-on-demand technolog...


Una vez eliminado los contenidos vacios se exporta el csv final

In [ ]:
modifiedcsv[['Titulo', 'ID', 'Autor','Descripcion']].to_csv('original_datasets/libros/goodreadsBooksModified.csv', index=False)

In [ ]:
libros = pd.read_csv('original_datasets/libros/goodreadsBooksModified.csv')
libros



,Titulo,ID,Autor,Descripcion
0,Harry Potter and the Half-Blood Prince (Harry ...,L1,[J.K. Rowling],The war against Voldemort is not going well; e...
1,Harry Potter and the Order of the Phoenix (Har...,L2,"[J.K. Rowling, Mary GrandPré]",There is a door at the end of a silent corrido...
2,Harry Potter and the Sorcerer's Stone (Harry P...,L3,"[J.K. Rowling, Mary GrandPré]",Harry Potter's life is miserable. His parents ...
3,Harry Potter and the Chamber of Secrets (Harry...,L4,[J.K. Rowling],The Dursleys were so mean and hideous that sum...
4,Harry Potter and the Prisoner of Azkaban (Harr...,L5,"[J.K. Rowling, Mary GrandPré]","For twelve long years, the dread fortress of A..."
5,Harry Potter and the Goblet of Fire (Harry Pot...,L6,"[J.K. Rowling, Mary GrandPré]",Harry Potter is midway through his training as...
6,"Harry Potter Boxed Set, Books 1-5 (Harry Potte...",L8,"[J.K. Rowling, Mary GrandPré]",Box Set containing Harry Potter and the Sorcer...
7,"Unauthorized Harry Potter Book Seven News: ""Ha...",L9,[W. Frederick Zimmerman],Through the magic of print-on-demand technolog...
